- Update: 2023.04.11

# 0. 配置参数

- `url`：CARiD.COM

In [1]:
url = 'https://www.carid.com/search/NGK+Laser+Iridium+Spark+Plugs/code-ccb8ce4eee843e4eb433d7a9a8931e0d/queryId-420738fe115bb7ad7af949d2688d7e1a/&redirect=1%2Fsort-price-asc#prod-list'

chromedriver = '../../chromedriver.exe'

In [2]:
from selenium.webdriver import ChromeOptions, Chrome
from selenium.webdriver.chrome.service import Service
import time

In [5]:
chrome_options = ChromeOptions()
chrome_options.add_experimental_option('excludeSwitches', ['enable-automation']) # Chrome正受到自动测试软件的控制
chrome_options.add_argument('--disable-blink-features=AutomationControlled') # window.navigator.webdriver=false
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36')
# chrome_options.add_argument('--proxy-server=http://c126.kdltps.com:15818')
# chrome_options.add_argument('blink-settings=imagesEnabled=false')
# chrome_options.page_load_strategy = 'eager'

driver = Chrome(service=Service(chromedriver), options=chrome_options)
driver.maximize_window()
driver.get(url)
time.sleep(0.3)

In [4]:
from selenium.webdriver.common.by import By
import pandas as pd

In [ ]:
try:
    while True:
        driver.find_element(By.XPATH, '//*[@class="simple-btn -large -wide -load-next -transparent js-pagination-more-btn"]').click()
        time.sleep(5)
except:
    pass

In [ ]:
list_product = ['https://www.carid.com' + i.get_attribute('data-url') for i in driver.find_element(By.XPATH, '//*[@data-ga-product-list-type="Search Results"]').find_elements(By.XPATH, '//*[@class="js-product-list-item"]')]
pd.DataFrame({'Product': list_product}).to_excel('./menu-ngk_laser_iridium_spark_plugs.xlsx', index=False)
print('总数量：' + str(len(list_product)))
print()

# 1. 爬取

In [ ]:
list_product = pd.read_excel('./menu-ngk_laser_iridium_spark_plugs.xlsx')['Product'].to_list()

list_product

In [ ]:
df = pd.DataFrame(columns=['Item',
                           '标题',
                           '图片',
                           'Brand',
                           'Part Number',
                           'UPC',
                           '车型',
                           'Center Electrode Material',
                           'Ground Electrode Material',
                           'Electrode Material',
                           '链接',
                           '图片链接'])

In [18]:
driver.get(list_product[i])
driver.maximize_window()
time.sleep(0.3)

In [19]:
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [20]:
WebDriverWait(driver, 30, 0.5, None).until(EC.presence_of_element_located((By.CLASS_NAME, 'prod-title.js-prod-title')))
title = driver.find_element(By.CLASS_NAME, 'prod-title.js-prod-title').find_element(By.TAG_NAME, 'h1').text
item = driver.find_element(By.CLASS_NAME, 'prod-title.js-prod-title').find_element(By.TAG_NAME, 'div').text.split('|')[1].replace('Item # ', '')
pic = driver.find_element(By.XPATH, '//*[@class="product-image-main"]').get_attribute('href')

print(item + '; ' + title + '; ' + pic)

mpn4673007080 NGK® 96107 - Spark Plug https://images.carid.com/ngk/items/96107.jpg


In [21]:
import re

In [23]:
try:
    specifications = driver.find_element(By.XPATH, '//*[@class="spec-stripped-table"]').text

    center_electrode_material = re.findall('\nCenter Electrode\nMaterial: .*?\n', specifications)[0].replace('Center Electrode\nMaterial: ', '').replace('\n', '')
    ground_electrode_material = re.findall('\nGround Electrode\nMaterial: .*?\n', specifications)[0].replace('Ground Electrode\nMaterial: ', '').replace('\n', '')
    electrode_material = re.findall('\nElectrode Material: .*?\n', specifications)[0].replace('Electrode Material: ', '').replace('\n', '')
except:
    center_electrode_material = ''
    ground_electrode_material = ''
    electrode_material = ''
    
print(center_electrode_material + '; ' + ground_electrode_material + '; ' + electrode_material)

In [ ]:
brand = ''
part_number = ''
upc = ''
WebDriverWait(driver, 30, 0.5, None).until(EC.presence_of_element_located((By.XPATH, '//*[@class="prod-offer-item"]')))
list_feature = driver.find_elements(By.XPATH, '//*[@class="prod-offer-item"]')
for feature in list_feature:
    if feature.text.startswith('Brand'):
        brand = feature.text.split(':')[1]
    elif feature.text.startswith('Part Number'):
        part_number = feature.text.split(':')[1]
    elif feature.text.startswith('UPC'):
        upc = feature.text.split(':')[1]

print(brand + '; ' + part_number + '; ' + upc)

In [ ]:
for j in driver.find_elements(By.XPATH, '//*[@class="mmy-row-expand"]')[1:]:
    j.click()
    time.sleep(0.5)

In [ ]:
list_make = [j.text for j in driver.find_elements(By.XPATH, '//*[@class="mmy-row-title pointer js-mmy-row-expand"]')]
list_model = [j.text for j in driver.find_elements(By.XPATH, '//*[@class="mmy-row-description"]')]

list_vehicle = []
for j in range(len(list_make)):
    list_vehicle.extend([list_make[j] + ' ' + k for k in list_model[j].split('\n')])
    
vehicle = '\n'.join(list_vehicle)

print(vehicle)

In [17]:
df_temp = pd.DataFrame({'Item': [item],
                        '标题': [title],
                        '图片': [''],
                        'Brand': [brand],
                        'Part Number': [part_number],
                        'UPC': [upc],
                        '车型': [vehicle],
                        'Center Electrode Material': [center_electrode_material],
                        'Ground Electrode Material': [ground_electrode_material],
                        'Electrode Material': [electrode_material],
                        '链接': [list_product[i]],
                        '图片链接': [pic]})

df_temp

NameError: name 'list_product' is not defined

In [ ]:
df = pd.concat([df, df_temp], ignore_index=True).fillna('')

driver.close()

df